# Model Training

In [128]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import random
from torchvision import transforms, models
import cv2
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
from torchvision import datasets

# Set up transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the full dataset
full_dataset = datasets.ImageFolder(root="data/train", transform=transform)

# Create indices for training and testing split
dataset_size = len(full_dataset)
indices = list(range(dataset_size))
train_indices, test_indices = train_test_split(indices, test_size=0.25, stratify=[full_dataset.targets[i] for i in indices])

# Create subsets
train_dataset = Subset(full_dataset, train_indices)
test_dataset = Subset(full_dataset, test_indices)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Checking the classes
print("Classes:", full_dataset.classes)

# Define the augmentation function to place segments on random locations
def place_segment_on_face(segment, canvas_size=(224, 224)):
    # Create an empty canvas (you can change the background to simulate skin tones)
    canvas = np.zeros((canvas_size[0], canvas_size[1], 3), dtype=np.uint8)

    # Randomly pick a position to place the segment
    x_offset = random.randint(0, canvas_size[0] - segment.shape[0])
    y_offset = random.randint(0, canvas_size[1] - segment.shape[1])

    # Place the segment on the canvas at the random location
    canvas[x_offset:x_offset + segment.shape[0], y_offset:y_offset + segment.shape[1]] = segment
    return canvas

# Augmentation transformations for segment-based images (cheeks, nose, etc.)
def augment_segment(segment):
    transform = transforms.Compose([
        transforms.ToPILImage(),  # Convert to PIL image for augmentation
        transforms.RandomRotation(15),  # Random rotation
        transforms.RandomHorizontalFlip(),  # Horizontal flip
        transforms.RandomVerticalFlip(),  # Vertical flip (if applicable)
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color tweak
        transforms.RandomCrop(224),  # Random crop to simulate varying positions
        transforms.ToTensor()  # Convert back to tensor
    ])
    return transform(segment)

# Define a custom dataset that applies augmentation to segments
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, segmented_images, labels, transform=None):
        self.segmented_images = segmented_images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.segmented_images)

    def __getitem__(self, idx):
        segment = self.segmented_images[idx]
        label = self.labels[idx]
        
        if self.transform:
            segment = self.transform(segment)  # Apply the augmentation to each segment
        
        return segment, label

# Calculate class weights using sklearn's compute_class_weight
train_labels = [label for _, label in train_loader.dataset]  # Assuming labels are directly accessible from the dataset
num_classes = len(set(train_labels))  # Number of classes (update as necessary)
classes = np.array([i for i in range(num_classes)])  # Convert to numpy array for class indices
class_weights = compute_class_weight('balanced', classes=classes, y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# Print class names and their corresponding weights
for i, weight in enumerate(class_weights):
    print(f"Class {i} Weight: {weight:.4f}")

# Load the pre-trained model and modify the final layer
model = models.shufflenet_v2_x0_5(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Define the criterion (loss function) with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
print("Starting training...")
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Evaluation function to calculate per-class accuracy and overall accuracy
def evaluate_per_class(model, test_loader):
    model.eval()  # Set to evaluation mode
    all_preds = []
    all_labels = []

    # Initialize a confusion matrix with zeros
    conf_matrix = np.zeros((len(test_loader.dataset.dataset.classes), len(test_loader.dataset.dataset.classes)))

    with torch.no_grad():  # No need to compute gradients during evaluation
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Send to device
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            # Update confusion matrix
            for t, p in zip(labels.view(-1), preds.view(-1)):
                conf_matrix[t.item(), p.item()] += 1

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate accuracy for each class
    class_accuracies = []
    for i in range(len(test_loader.dataset.dataset.classes)):
        class_correct = conf_matrix[i, i]
        class_total = conf_matrix[i].sum()
        class_accuracy = class_correct / class_total if class_total > 0 else 0
        class_accuracies.append(class_accuracy)
        print(f"Accuracy for class {test_loader.dataset.dataset.classes[i]}: {class_accuracy * 100:.2f}%")

    # Calculate overall accuracy
    accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    return accuracy, class_accuracies

# Get the class-wise accuracy and overall accuracy
overall_accuracy, class_accuracies = evaluate_per_class(model, test_loader)
print(f"Overall Test Accuracy: {overall_accuracy * 100:.2f}%")


Classes: ['Blackheads', 'Clear Skin', 'Cystic', 'Papules', 'Pustules', 'Rosacea', 'Whiteheads']


KeyboardInterrupt: 

### Save Model

In [124]:
# Saving
model_save_path = 'model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to model.pth


### Load Model

In [129]:
# Loading
model = models.shufflenet_v2_x0_5(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_save_path))
model = model.to(device)
model.eval()
print("Model loaded.")

Model loaded.


c:\Users\Nathan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Nathan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_17168\1879304336.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be fli

# RL Testing

In [131]:
import torch
from PIL import Image
from torchvision import transforms, models

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model (make sure you have loaded the weights)
model = models.shufflenet_v2_x0_5(pretrained=False)  # Use the model architecture you trained
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # Modify the final layer as needed
model.load_state_dict(torch.load('model.pth'))  # Load the saved model weights
model = model.to(device)  # Move the model to the correct device
model.eval()  # Set the model to evaluation mode

# Replace 'your_image.jpg' with the path to the image you want to classify
image_path = 'data/test/image.jpg'

# Define the image transformations (resize, normalization) for model input
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224 for the model
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load and preprocess the image
image = Image.open(image_path)
image = transform(image).unsqueeze(0)  # Add a batch dimension (1, C, H, W)

# Move the image tensor to the same device as the model
image = image.to(device)

# Perform inference (forward pass)
with torch.no_grad():  # Disable gradient computation during inference
    outputs = model(image)

# Get the predicted class (index of the highest logit)
_, predicted_class = torch.max(outputs, 1)

# Access the classes from the original dataset (ImageFolder or similar)
# Make sure to access the original dataset from train_loader (not the subset)
classes = train_loader.dataset.dataset.classes  # Assuming your loader is a subset
class_label = classes[predicted_class.item()]

print(f"The image is classified as: {class_label}")


The image is classified as: Pustules


C:\Users\Nathan\AppData\Local\Temp\ipykernel_17168\568179294.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))  # Load the 